# KeyPoints Model

In [ ]:
import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import matplotlib.pyplot as plt

In [ ]:
keyfacial_df = pd.read_csv("archive (1)/training/training.csv")

In [ ]:
keyfacial_df.head(5)

In [ ]:
keyfacial_df.isnull().sum()

In [ ]:
keyfacial_df.dropna(how = 'any', inplace = True)

In [ ]:
keyfacial_df.isnull().sum()

In [ ]:
keyfacial_df.shape

In [ ]:
keyfacial_df['Image'] = keyfacial_df['Image'].apply(lambda x: np.fromstring(x, dtype = int, sep = ' ').reshape(96, 96))

In [ ]:
keyfacial_df['Image'][0].shape

In [ ]:
keyfacial_df.describe()

In [ ]:
i = np.random.randint(1, len(keyfacial_df))
plt.imshow(keyfacial_df['Image'][i], cmap = 'gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df.loc[i][j-1], keyfacial_df.loc[i][j], 'rx')

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1)    
    image = plt.imshow(keyfacial_df['Image'][i],cmap = 'gray')
    for j in range(1,31,2):
        plt.plot(keyfacial_df.loc[i][j-1], keyfacial_df.loc[i][j], 'rx')

In [ ]:
import copy
keyfacial_df_copy = copy.copy(keyfacial_df)

In [ ]:
columns = keyfacial_df_copy.columns[:-1]
columns

In [ ]:
keyfacial_df_copy['Image'] = keyfacial_df_copy['Image'].apply(lambda x: np.flip(x, axis = 1))

In [ ]:
for i in range(len(columns)):
  if i%2 == 0:
    keyfacial_df_copy[columns[i]] = keyfacial_df_copy[columns[i]].apply(lambda x: 96. - float(x) )

In [ ]:
plt.imshow(keyfacial_df['Image'][0], cmap = 'gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df.loc[0][j-1], keyfacial_df.loc[0][j], 'rx')

In [ ]:
plt.imshow(keyfacial_df_copy['Image'][0],cmap='gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df_copy.loc[0][j-1], keyfacial_df_copy.loc[0][j], 'rx')

In [ ]:
augmented_df = np.concatenate((keyfacial_df, keyfacial_df_copy))

In [ ]:
import random

keyfacial_df_copy = copy.copy(keyfacial_df)
keyfacial_df_copy['Image'] = keyfacial_df_copy['Image'].apply(lambda x:np.clip(random.uniform(1.5, 2)* x, 0.0, 255.0))
augmented_df = np.concatenate((augmented_df, keyfacial_df_copy))
augmented_df.shape

In [ ]:
plt.imshow(keyfacial_df_copy['Image'][0], cmap='gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df_copy.loc[0][j-1], keyfacial_df_copy.loc[0][j], 'rx')

In [ ]:
keyfacial_df_copy = copy.copy(keyfacial_df)

In [ ]:
keyfacial_df_copy['Image'] = keyfacial_df_copy['Image'].apply(lambda x: np.flip(x, axis = 0))

for i in range(len(columns)):
  if i%2 == 1:
    keyfacial_df_copy[columns[i]] = keyfacial_df_copy[columns[i]].apply(lambda x: 96. - float(x) )

In [ ]:
plt.imshow(keyfacial_df_copy['Image'][0], cmap='gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df_copy.loc[0][j-1], keyfacial_df_copy.loc[0][j], 'rx')

In [ ]:
img = augmented_df[:,30]

# Normalize the images
img = img/255.

# Create an empty array of shape (x, 96, 96, 1) to feed the model
X = np.empty((len(img), 96, 96, 1))

# Iterate through the list and add image values to the empty array after expanding it's dimension from (96, 96) to (96, 96, 1)
for i in range(len(img)):
  X[i,] = np.expand_dims(img[i], axis = 2)

# Convert the array type to float32
X = np.asarray(X).astype(np.float32)
X.shape

In [ ]:
y = augmented_df[:,:30]
y = np.asarray(y).astype(np.float32)
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend
from keras import optimizers

In [ ]:
def res_block(X, filter, stage):

  # Convolutional_block
  X_copy = X

  f1 , f2, f3 = filter

  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = MaxPool2D((2,2))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)


  # Short path
  X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(X_copy)
  X_copy = MaxPool2D((2,2))(X_copy)
  X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  # Identity Block 1
  X_copy = X


  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  # Identity Block 2
  X_copy = X


  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  return X

In [ ]:
input_shape = (96, 96, 1)

# Input tensor shape
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((3,3))(X_input)

# 1 - stage
X = Conv2D(64, (7,7), strides= (2,2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides= (2,2))(X)

# 2 - stage
X = res_block(X, filter= [64,64,256], stage= 2)

# 3 - stage
X = res_block(X, filter= [128,128,512], stage= 3)


# Average Pooling
X = AveragePooling2D((2,2), name = 'Averagea_Pooling')(X)

# Final layer
X = Flatten()(X)
X = Dense(4096, activation = 'relu')(X)
X = Dropout(0.2)(X)
X = Dense(2048, activation = 'relu')(X)
X = Dropout(0.1)(X)
X = Dense(30, activation = 'relu')(X)


In [ ]:
model_1_facialKeyPoints = Model( inputs= X_input, outputs = X)
model_1_facialKeyPoints.summary()

In [ ]:
import pydot

In [ ]:
#tf.keras.utils.plot_model(model_1_facialKeyPoints, 'facial_model.png', show_shapes = True, show_layer_names = True)

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
model_1_facialKeyPoints.compile(loss = "mean_squared_error", optimizer = adam , metrics = ['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(filepath = "FacialKeyPoints_weights.hdf5", verbose = 1, save_best_only = True)

In [ ]:
earlystopping = EarlyStopping( monitor = 'val_accuracy', patience= 3, mode = 'max' , restore_best_weights= True , min_delta= 0.001 )

In [ ]:
callbacks = [checkpointer]

In [ ]:
history = model_1_facialKeyPoints.fit(X_train, y_train, batch_size = 32, epochs = 30, validation_split = 0.05, callbacks=callbacks)

In [ ]:
model = load_model("FacialKeyPoints_weights.hdf5")

In [ ]:
model_json = model.to_json()
with open("FacialKeyPoints-model.json","w") as json_file:
  json_file.write(model_json)
model.save_weights("FacialKeyPoints-model.h5")

# Emotion Model

In [ ]:
facialexpression_df = pd.read_csv("challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv")

In [ ]:
facialexpression_df.head(5)

In [ ]:
facialexpression_df.drop(labels = ' Usage', axis = 1, inplace = True)

In [ ]:
facialexpression_df.isnull().sum()

In [ ]:
facialexpression_df[' pixels'][0] 

In [ ]:
def string2array(x):
    return np.array(x.split(' ')).reshape(48, 48, 1).astype('float32')

In [ ]:
def resize(x):
  img = x.reshape(48, 48)
  return cv2.resize(img, dsize=(96, 96), interpolation = cv2.INTER_CUBIC)

In [ ]:
facialexpression_df[' pixels'] = facialexpression_df[' pixels'].apply(lambda x: string2array(x))

In [ ]:
facialexpression_df[' pixels'] = facialexpression_df[' pixels'].apply(lambda x: resize(x))

In [ ]:
facialexpression_df.head(5)

In [ ]:
facialexpression_df.shape

In [ ]:
label_to_text = {0:'anger', 1:'disgust',2:'fear', 3:'happy', 4:'sad', 5: 'surprise', 6:'neutral'}

In [ ]:
plt.imshow(facialexpression_df[' pixels'][0], cmap = 'gray')

In [ ]:
emotions = [0, 1, 2, 3, 4, 5, 6]

for i in emotions:
  data = facialexpression_df[facialexpression_df['emotion'] == i][:1]
  img = data[' pixels'].item()
  img = img.reshape(96, 96)
  plt.figure()
  plt.title(label_to_text[i])
  plt.imshow(img, cmap = 'gray')

In [ ]:
facialexpression_df.emotion.value_counts().index

In [ ]:
facialexpression_df.emotion.value_counts()

In [ ]:
plt.figure(figsize = (10,10))
sns.countplot(x = facialexpression_df.emotion)

In [ ]:
from tensorflow.keras.utils import to_categorical

X = facialexpression_df[' pixels']
y = to_categorical(facialexpression_df['emotion'])

In [ ]:
X = np.stack(X, axis = 0)
X = X.reshape(35887, 96, 96, 1)

print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, shuffle = True)

In [ ]:
X_train = X_train/255
X_val   = X_val /255
X_test  = X_test/255

In [ ]:
train_datagen = ImageDataGenerator(
rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = "nearest")

In [ ]:
input_shape = (96, 96, 1)

# Input tensor shape
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((3, 3))(X_input)

# 1 - stage
X = Conv2D(64, (7, 7), strides= (2, 2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides= (2, 2))(X)

# 2 - stage
X = res_block(X, filter= [64, 64, 256], stage= 2)

# 3 - stage
X = res_block(X, filter= [128, 128, 512], stage= 3)

# 4 - stage
# X = res_block(X, filter= [256, 256, 1024], stage= 4)

# Average Pooling
X = AveragePooling2D((4, 4), name = 'Averagea_Pooling')(X)

# Final layer
X = Flatten()(X)
X = Dense(7, activation = 'softmax', name = 'Dense_final', kernel_initializer= glorot_uniform(seed=0))(X)


In [ ]:
model_2_emotion = Model( inputs= X_input, outputs = X, name = 'Resnet18')
model_2_emotion.summary()

In [ ]:
model_2_emotion.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)

# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath = "FacialExpression_weights.hdf5", verbose = 1, save_best_only=True)

In [ ]:
history1 = model_2_emotion.fit(train_datagen.flow(X_train, y_train, batch_size=64),
                               validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // 64,
                               epochs= 25, callbacks=[checkpointer, earlystopping])

In [ ]:
model_json = model_2_emotion.to_json()
with open("FacialExpression-model.json","w") as json_file:
  json_file.write(model_json)
model.save_weights("FacialExpression-model.h5")

In [ ]:
score = model_2_emotion.evaluate(X_test, y_test)
print('Test Accuracy: {}'.format(score[1]))

In [ ]:
predicted_classes = np.argmax(model_2_emotion.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, predicted_classes)
plt.figure(figsize = (10, 10))
sns.heatmap(cm, annot = True, cbar = False)

In [ ]:
L = 5
W = 5

fig, axes = plt.subplots(L, W, figsize = (24, 24))
axes = axes.ravel()

for i in np.arange(0, L*W):
    axes[i].imshow(X_test[i].reshape(96,96), cmap = 'gray')
    axes[i].set_title('Prediction = {}\n True = {}'.format(label_to_text[predicted_classes[i]], label_to_text[y_true[i]]))
    axes[i].axis('off')

plt.subplots_adjust(wspace = 1)  

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, predicted_classes))

In [ ]:
model = load_model('FacialKeyPoints_weights.hdf5')

In [ ]:
def predict(X_test):
    df_predict = model_1_facialKeyPoints.predict(X_test)

  # Making prediction from the emotion model
    df_emotion = np.argmax(model_2_emotion.predict(X_test), axis=-1)

  # Reshaping array from (856,) to (856,1)
    df_emotion = np.expand_dims(df_emotion, axis = 1)

  # Converting the predictions into a dataframe
    df_predict = pd.DataFrame(df_predict, columns= columns)

  # Adding emotion into the predicted dataframe
    df_predict['emotion'] = df_emotion
    return df_predict

In [ ]:
df_predict = predict(X_test)

In [ ]:
fig, axes = plt.subplots(4, 4, figsize = (24, 24))
axes = axes.ravel()

for i in range(16):

    axes[i].imshow(X_test[i].squeeze(),cmap='gray')
    axes[i].set_title('Prediction = {}'.format(label_to_text[df_predict['emotion'][i]]))
    axes[i].axis('off')
    for j in range(1,31,2):
            axes[i].plot(df_predict.loc[i][j-1], df_predict.loc[i][j], 'rx')

# Prediction

In [1]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
from PIL import Image

In [2]:
import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend
from keras import optimizers

In [4]:
label_to_text = {0:'Anger', 1:'Disgust',2:'Fear', 3:'Happy', 4:'Sad', 5: 'Surprise', 6:'Neutral'}

In [5]:
face_haar_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

model_1 = load_model('FacialKeyPoints_weights.hdf5')

model_2 = load_model('FacialExpression_weights.hdf5')

In [10]:
def predict(X_test):
    
    df_predict = model_1.predict(X_test)
    print("df_predict: {}".format(df_predict))

  # Making prediction from the emotion model
    df_emotion = np.argmax(model_2.predict(X_test), axis=-1)

  # Reshaping array from (856,) to (856,1)
    df_emotion = np.expand_dims(df_emotion, axis = 1)

  # Converting the predictions into a dataframe
    df_predict = pd.DataFrame(df_emotion, columns = ['A'])
    
    print("Data Frame: {}".format(df_predict))
    #print("Size of data frame: {}".format(df_predict.shape))
    
    return df_predict

In [12]:
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

In [13]:
rectangle_bgr = (255,255,255)
img = np.zeros((500,500)) #makes a black image
text = "Some text in a box!"
(text_width, text_height) = cv2.getTextSize(text, font, fontScale = font_scale, thickness=1)[0]
text_offset_x = 10
text_offset_y = img.shape[0] - 25
box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height - 2))
cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale=font_scale, color=(0,0,0), thickness=1)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray,1.1,4)
    
    for x,y,w,h in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
    
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        face = face_haar_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in face:
            face_roi = roi_color[ey:ey+eh, ex:ex+ew]
                
            final_image = cv2.resize(face_roi, (96,96))
            final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2GRAY)
            final_image = np.expand_dims(final_image,axis=0)
            final_image = final_image/255.0

            font = cv2.FONT_HERSHEY_SIMPLEX

            df_predict = predict(final_image)

            emotion_prediction = df_predict.at[0,'A']
        
            if emotion_prediction == 0:
                emo = "Angry"
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
                cv2.putText(frame, emo, (x1+int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, emo, (100,150), font, 3, (0,0,255), 2, cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255))
            
            elif emotion_prediction == 1:
                emo = "Disgust"
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
                cv2.putText(frame, emo, (x1+int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, emo, (100,150), font, 3, (0,0,255), 2, cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255))
            
            elif emotion_prediction == 2:
                emo = "Fear"
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
                cv2.putText(frame, emo, (x1+int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, emo, (100,150), font, 3, (0,0,255), 2, cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255))
            
            elif emotion_prediction == 3:
                emo = "Happy"
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
                cv2.putText(frame, emo, (x1+int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, emo, (100,150), font, 3, (0,0,255), 2, cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255))
            
            elif emotion_prediction == 4:
                emo = "Sad"
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
                cv2.putText(frame, emo, (x1+int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, emo, (100,150), font, 3, (0,0,255), 2, cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255))
            
            elif emotion_prediction == 5:
                emo = "Surprised"
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
                cv2.putText(frame, emo, (x1+int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, emo, (100,150), font, 3, (0,0,255), 2, cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255))
            
            else:
                emo = "Neutral"
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
                cv2.putText(frame, emo, (x1+int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, emo, (100,150), font, 3, (0,0,255), 2, cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255))

    cv2.imshow("Image", frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

df_predict: [[57.24128  34.600136 36.039474 32.91991  53.731716 35.70739  62.202927
  35.762024 40.335594 34.341484 31.840477 33.828228 53.758045 26.212645
  64.984665 26.01433  43.792984 25.830778 28.504095 24.852726 48.44133
  58.194878 55.439014 76.17973  35.987644 74.99651  45.983204 74.711685
  46.618244 79.795586]]
Data Frame:    A
0  5
df_predict: [[61.896343 35.212814 32.225708 32.96289  56.801212 36.21939  67.97721
  36.617973 37.440502 34.644764 26.391407 34.153324 55.922802 26.793486
  72.73597  27.20845  41.299904 25.966995 21.92853  25.557285 46.731236
  57.41668  57.77554  76.2972   31.755001 74.42786  44.832966 73.69394
  45.25753  80.24246 ]]
Data Frame:    A
0  6
df_predict: [[55.194763 36.512577 38.691143 33.76432  52.192505 37.011497 58.97026
  37.979107 41.46547  35.34182  35.47069  34.781605 52.204124 27.849936
  61.06619  28.77186  44.187428 26.65022  33.120815 26.171835 46.25707
  58.798397 53.375313 79.33742  38.01352  77.84568  45.363323 76.08816
  45.53456  83

df_predict: [[57.81198  36.460873 37.504044 34.5683   54.35055  37.509693 61.934097
  37.78591  41.147243 35.864014 33.456223 35.822052 54.040657 28.064104
  65.522545 28.880081 44.835373 27.596283 30.721817 27.371777 47.67139
  58.281017 54.503445 78.81628  36.554935 77.45578  45.579334 75.487144
  46.017666 83.42749 ]]
Data Frame:    A
0  6
df_predict: [[42.387016 36.349606 51.99642  34.150005 44.29514  37.45709  40.60803
  37.651344 50.242462 35.553017 54.08379  34.93073  46.533215 28.324461
  38.666122 28.900427 50.880165 27.643894 56.24166  26.742231 47.12323
  58.867275 42.13174  79.18406  49.38507  77.57609  45.73125  76.437614
  45.92785  83.29306 ]]
Data Frame:    A
0  6
df_predict: [[58.402775 37.249416 36.494637 34.935074 54.667747 37.888023 62.871986
  38.493996 40.213417 36.33103  32.264004 36.36894  54.3597   29.00637
  66.78623  29.872963 44.18733  28.250502 28.661842 27.997145 47.360466
  58.756603 55.119125 79.588875 35.45428  78.10543  45.400246 75.90481
  45.64544  8

df_predict: [[48.25236  35.56954  45.40435  33.641647 48.10651  36.963757 48.895424
  36.86102  46.02966  35.31136  45.09035  34.485184 49.897873 27.206905
  48.939583 27.35206  48.22949  26.824522 45.33151  25.638124 47.54997
  58.554092 46.29676  78.25378  43.647736 76.78615  45.221916 76.60733
  45.455982 81.25073 ]]
Data Frame:    A
0  6
df_predict: [[54.61397  35.734417 40.46943  33.846157 52.53218  36.79798  57.664738
  36.933647 43.158947 35.39562  37.888474 34.92458  53.11649  27.10335
  59.878437 27.59577  46.44512  26.756147 35.986195 26.029964 48.50411
  58.503937 51.93228  77.661804 39.249886 76.36338  45.766266 75.67805
  46.37905  81.1453  ]]
Data Frame:    A
0  6
df_predict: [[50.9796   35.96831  43.33724  33.94601  50.171326 37.112797 52.727955
  37.010834 44.89119  35.71428  42.04164  35.01055  51.192154 27.575008
  53.741104 27.868885 47.697727 27.251287 41.078617 26.385357 48.57713
  58.719685 49.193172 78.00825  41.972664 76.50042  45.92296  75.89462
  46.232327 81.

Data Frame:    A
0  6
df_predict: [[48.46785  34.773556 45.28865  32.88049  48.295258 35.94672  49.484715
  36.14412  45.962856 34.40807  44.460903 33.827312 49.90911  25.76771
  49.620865 26.272741 48.645294 25.544228 44.39139  24.55991  47.917816
  57.241497 46.591324 76.76471  43.07443  75.36262  45.302544 74.5934
  45.55763  80.1297  ]]
Data Frame:    A
0  6
df_predict: [[60.07519  35.44122  34.525326 33.849438 55.53863  36.64348  65.27953
  36.60327  39.127758 35.26022  29.2575   34.97246  55.60775  27.173822
  69.36353  27.329569 43.179626 26.860231 25.76144  26.093126 48.05996
  58.461323 56.305405 77.50453  34.09745  76.05334  45.472534 75.56546
  45.99019  81.29666 ]]
Data Frame:    A
0  5
df_predict: [[45.664616 35.105736 48.326813 33.709496 46.40785  36.428627 45.60503
  36.57923  48.207554 35.157703 49.04683  34.625706 48.282597 26.254438
  44.503315 26.62222  49.78637  26.16925  49.47088  25.367165 48.41644
  58.158268 45.314575 78.12002  46.749073 77.04066  46.355938 76.0

df_predict: [[46.10373  34.613945 47.76556  33.15718  46.56453  36.21839  46.069714
  35.921875 47.979347 34.761616 48.358646 33.988144 48.659397 26.431103
  45.073692 26.19551  49.164707 26.413548 48.93751  25.169167 48.22602
  58.59577  45.57308  77.27678  46.3705   76.170166 46.14698  76.09488
  46.65148  80.51049 ]]
Data Frame:    A
0  6
df_predict: [[43.15504  34.758945 50.614563 33.190277 44.883507 36.347244 41.95451
  35.998962 49.67327  34.634045 52.12507  33.95419  47.045406 26.62859
  40.000347 26.407877 50.61048  26.569794 53.850086 25.149303 48.39008
  58.52232  43.13778  77.301384 48.598244 75.99128  46.10388  76.09272
  46.614586 80.41055 ]]
Data Frame:    A
0  5
df_predict: [[60.149548 34.65697  34.40905  33.257786 55.26298  35.830696 65.651695
  35.76916  39.24504  34.72991  28.69312  34.534126 55.034874 26.050236
  69.64736  26.098999 43.090305 25.974098 24.604277 25.405602 48.239487
  57.25856  57.16425  76.217834 33.90813  75.10991  45.981792 74.36559
  46.46038  79.

...

df_predict: [[39.857834 35.812    52.782482 33.99686  42.41323  37.00425  37.461758
  36.868156 50.518826 35.16227  54.83989  34.65179  44.868023 28.219055
  35.071434 28.357372 50.992146 27.820574 57.61132  26.775873 47.51075
  59.275322 40.605618 77.94697  49.89803  76.46614  45.9153   75.88994
  46.104954 81.83291 ]]
Data Frame:    A
0  6
df_predict: [[51.581104 35.58762  41.690075 33.702694 50.672768 36.436302 53.904808
  36.429253 43.667873 35.133827 39.96785  34.66893  51.559338 27.35312
  55.324455 27.523409 46.61453  27.226446 38.47802  26.280426 48.107098
  57.80561  49.75019  76.66317  40.669716 75.462715 45.433666 74.787315
  45.93291  80.062546]]
Data Frame:    A
0  3
df_predict: [[37.77608  34.682156 53.890476 33.407276 40.986477 35.914215 34.778393
  35.721184 51.02547  34.569286 57.050632 33.94648  43.64632  27.265293
  31.454544 26.952574 50.637783 27.248232 59.972946 26.097265 46.930458
  57.206757 38.868896 74.778496 51.434944 73.846504 45.08638  73.45233
  45.905155 

df_predict: [[45.22185  34.643974 47.05567  33.19796  45.87659  35.958973 45.26746
  35.768097 47.26008  34.57638  47.49665  34.011665 47.686954 26.972036
  44.392406 26.5316   48.40781  26.961222 47.838364 25.470766 47.638412
  58.286873 45.08522  76.229385 45.781475 75.09645  45.47222  75.22148
  46.15916  79.42934 ]]
Data Frame:    A
0  3
df_predict: [[35.800808 35.214756 56.79893  33.90429  39.75948  36.272934 31.77016
  36.095303 53.294296 34.96811  60.609272 34.326042 42.773388 27.7019
  27.792496 27.423445 52.362946 27.615582 64.90414  26.385347 47.149475
  58.40683  37.277615 76.397575 53.722466 75.55686  45.45114  74.8713
  46.300472 80.57752 ]]
Data Frame:    A
0  6
df_predict: [[46.61101  35.26496  46.086327 33.39265  47.11753  36.48555  46.877617
  36.375492 46.41737  34.823895 46.230404 34.475933 48.684547 27.05633
  46.62561  27.27155  48.34835  26.814243 46.588593 25.924225 47.66224
  57.232124 45.118732 76.4609   44.230057 75.163864 45.141014 74.38029
  45.333496 80.097

df_predict: [[33.367847 34.355778 57.587395 33.190804 38.04191  35.305084 28.92536
  35.519325 53.484447 34.00176  62.39418  33.650852 40.91521  26.770145
  24.261337 26.795048 52.056816 26.603222 66.379265 25.874022 46.649014
  56.778515 36.17958  74.60054  55.20375  74.12505  45.294697 72.77511
  46.43208  78.65216 ]]
Data Frame:    A
0  6
df_predict: [[44.236507 35.18946  49.100792 33.593307 45.567196 36.41023  43.888496
  36.11969  48.698322 35.048203 50.30971  34.31114  47.71694  27.3341
  42.285107 26.871824 49.798782 27.051651 50.953995 25.486843 48.440105
  58.716858 44.32845  77.1278   47.544827 75.86919  46.347427 75.95327
  46.87853  80.149345]]
Data Frame:    A
0  3
df_predict: [[48.598045 36.423325 45.600353 34.58457  48.59976  37.586582 49.35209
  37.301575 45.767826 36.003826 45.047638 35.575222 49.681416 29.166756
  49.78412  28.741545 48.066147 28.631765 44.435913 27.35038  47.624184
  59.21745  47.647785 77.78614  44.276497 76.73607  46.04065  76.44116
  46.23198  81.

df_predict: [[48.847546 35.17381  45.2474   33.53478  48.556637 36.370205 50.124046
  36.05942  46.223614 34.952976 44.43464  34.491108 49.70444  26.753065
  50.045864 26.562164 47.945778 26.537937 44.1105   25.539587 48.401943
  58.269703 48.198696 76.62568  43.796604 75.50436  46.108223 75.27814
  46.647823 80.01134 ]]
Data Frame:    A
0  5
df_predict: [[40.65386  34.881584 52.083443 33.25096  43.15561  36.203617 38.749657
  36.014736 50.33934  34.5448   54.231956 33.970947 46.089897 26.71875
  36.245857 26.512394 51.129284 26.565914 56.54387  24.844086 48.346905
  58.33012  41.038025 77.16883  49.62971  75.89701  45.993847 75.599915
  46.29242  80.08354 ]]
Data Frame:    A
0  5
df_predict: [[41.77917  36.56182  52.528446 34.632572 43.843822 37.822327 39.541626
  37.67618  50.47009  36.010265 54.623917 35.57079  46.43009  28.308409
  37.518276 28.651747 51.177147 27.992765 57.1537   27.265265 47.797318
  59.341873 41.660454 79.946945 49.809128 78.773384 46.145893 77.43298
  46.438763

df_predict: [[43.923653 35.18309  48.367325 33.017178 44.99023  36.470684 43.058823
  36.306778 47.64889  34.636692 49.26258  33.79692  47.3011   27.034325
  41.74042  27.14898  49.028816 26.557375 50.830723 25.172388 47.077644
  57.75891  42.53486  77.2099   45.92814  75.463745 44.64645  74.828606
  44.850147 80.729706]]
Data Frame:    A
0  6
df_predict: [[61.84356  36.368576 32.749184 34.49697  56.797142 37.303047 67.52486
  37.50148  37.636448 35.6995   26.842726 35.88227  55.94769  28.057272
  72.55815  28.655554 42.071274 27.198696 22.30604  27.266014 47.841614
  58.44316  57.876892 78.51486  32.69999  76.792175 45.61667  75.33646
  46.13892  82.98602 ]]
Data Frame:    A
0  6
df_predict: [[52.946884 35.431904 40.84787  33.53625  50.96653  36.5094   55.89038
  36.459034 43.388535 34.969814 38.489964 34.52375  51.377434 27.331884
  57.22911  27.472475 45.89588  27.057861 36.84558  26.26921  47.927143
  58.098484 51.636917 77.51837  40.449085 76.305214 46.313934 75.10221
  46.672096 